# Architecture Lakehouse - NYC Taxi Data Portfolio

# Pipeline ETL - Step by Step

## Étape 1: Initialisation Spark avec Iceberg

In [2]:
# notebooks/01_setup_iceberg.py
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Configuration Spark avec Iceberg
spark = SparkSession.builder.appName("NYC_Taxi").config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Même catalog "iceberg" que Trino, en REST sur MinIO
spark.conf.set("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
spark.conf.set("spark.sql.catalog.iceberg.uri", "http://rest:8181")
spark.conf.set("spark.sql.catalog.iceberg.warehouse", "s3://warehouse/")

# Accès S3 / MinIO identique au service REST et à Trino
spark.conf.set("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
spark.conf.set("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000")
spark.conf.set("spark.sql.catalog.iceberg.s3.path-style-access", "true")
spark.conf.set("spark.sql.catalog.iceberg.s3.access-key-id", "minioadmin")
spark.conf.set("spark.sql.catalog.iceberg.s3.secret-access-key", "minioadmin123")
spark.conf.set("spark.sql.catalog.iceberg.s3.region", "us-east-1")



26/01/05 11:04:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[]

## Étape 2: Ingestion des Données Brutes (Bronze)

### Installation des dependances SIG

In [3]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.7/341.7 kB 980.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.4/32.4 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.7 MB/s eta 0:00:0000:0100:010m

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [8]:
# Création des namespaces Iceberg
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.bronze")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.silver")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.gold")

DataFrame[]

### creation des tables bronzes

In [10]:
from pyspark.sql import functions as F

def create_bronze_tables():
    """Crée les tables bronze depuis les données sources"""
    
    # 1. Taxi Trips (Parquet 2024)
    print("📊 Étape 1/3: Ingestion des trips taxi...")
    trips_df = spark.read.parquet("/home/iceberg/data/raw/nyc_taxi/taxi_trips/*.parquet")
    
    # Vérifier le schéma des trips
    print("Schéma des trips:")
    trips_df.printSchema()
    
    # Ajout des colonnes de partition
    trips_df = trips_df.withColumn("year", F.year("tpep_pickup_datetime")) \
                      .withColumn("month", F.month("tpep_pickup_datetime")) \
                      .withColumn("day", F.dayofmonth("tpep_pickup_datetime"))
    
    # Repartitionner pour l'écriture (moins de partitions pour éviter OOM)
    trips_df = trips_df.repartition(12, "year", "month")  # 12 partitions pour 12 mois

    # Vérifier les valeurs de partition
    print("\n📈 Distribution par année/mois:")
    trips_df.groupBy("year", "month").count().orderBy("year", "month").show()
    
    # Création table Iceberg avec partitions
    trips_df.writeTo("iceberg.bronze.taxi_trips_2024") \
            .partitionedBy("year", "month") \
            .createOrReplace()
    
    print(f"✅ Table bronze.taxi_trips_2024 créée: {trips_df.count()} lignes")
    
    # 2. Taxi Zones (Shapefile)
    print("\n🗺️  Étape 2/3: Ingestion des zones taxi...")
    
    # D'abord essayer de lire un CSV si disponible
    try:
        # Vérifier si un fichier CSV existe
        import os
        csv_path = "/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.csv"
        
        if os.path.exists(csv_path):
            zones_df = spark.read.option("header", "true").csv(csv_path)
            print("📁 Fichier CSV trouvé, chargement...")
        else:
            # Sinon utiliser le shapefile
            import geopandas as gpd
            import zipfile
            
            print("📁 Fichier CSV non trouvé, utilisation du shapefile...")
            # Extraction du shapefile
            with zipfile.ZipFile("/home/iceberg/data/raw/nyc_taxi/taxi_zones/taxi_zones.zip", 'r') as zip_ref:
                zip_ref.extractall("/tmp/taxi_zones")
            
            # Lecture avec geopandas
            gdf = gpd.read_file("/tmp/taxi_zones/taxi_zones.shp")
            
            # Conversion en DataFrame Spark
            zones_df = spark.createDataFrame(gdf.drop(columns='geometry'))
            
            # Ajout des colonnes WKT pour le géospatial (optionnel)
            zones_df = zones_df.withColumn("geometry_wkt", 
                                          F.lit(gdf.geometry.to_wkt().tolist()))
    except Exception as e:
        print(f"⚠️  Erreur avec les zones: {e}")
        print("Création d'une table zones vide...")
        from pyspark.sql.types import StructType, StructField, IntegerType, StringType
        
        schema = StructType([
            StructField("LocationID", IntegerType(), True),
            StructField("zone", StringType(), True),
            StructField("borough", StringType(), True)
        ])
        zones_df = spark.createDataFrame([], schema)
    
    # Afficher le schéma des zones
    print("Schéma des zones:")
    zones_df.printSchema()
    
    # Repartitionner en 1 pour éviter trop de fichiers
    zones_df = zones_df.repartition(1)
    
    zones_df.writeTo("iceberg.bronze.taxi_zones").createOrReplace()
    print(f"✅ Table bronze.taxi_zones créée: {zones_df.count()} zones")
    
    # 3. Données Météo
    print("\n🌤️  Étape 3/3: Ingestion des données météo...")
    
    weather_df = spark.read.option("header", "true") \
                          .option("inferSchema", "true") \
                          .csv("/home/iceberg/data/raw/nyc_taxi/weather/central_park_weather_2024.csv")
    
    # Afficher le schéma pour voir les colonnes disponibles
    print("Schéma des données météo:")
    weather_df.printSchema()
    print("Colonnes disponibles:", weather_df.columns)
    print("Aperçu des données:")
    weather_df.show(5)
    
    # CORRECTION : La colonne s'appelle probablement 'time' au lieu de 'datetime'
    # Vérifier quelle colonne contient la date
    if 'time' in weather_df.columns:
        # Si la colonne s'appelle 'time', la convertir en date
        weather_df = weather_df.withColumn("date", F.to_date("time"))
        print("✅ Colonne 'time' convertie en 'date'")
    elif 'date' in weather_df.columns:
        # Si une colonne 'date' existe déjà
        weather_df = weather_df.withColumnRenamed("date", "date_temp").withColumn("date", F.to_date("date_temp"))
        print("✅ Colonne 'date' déjà présente")
    else:
        # Essayer d'utiliser la première colonne qui ressemble à une date
        date_cols = [col for col in weather_df.columns if any(x in col.lower() for x in ['date', 'time', 'datetime', 'timestamp'])]
        if date_cols:
            weather_df = weather_df.withColumn("date", F.to_date(date_cols[0]))
            print(f"✅ Colonne '{date_cols[0]}' utilisée comme date")
        else:
            # Si aucune colonne de date n'est trouvée, créer une date par défaut
            weather_df = weather_df.withColumn("date", F.lit("2024-01-01").cast("date"))
            print("⚠️  Aucune colonne de date trouvée, date par défaut utilisée")
    
    # Repartitionner par date pour l'écriture
    weather_df = weather_df.repartition(12, "date")  # 12 partitions pour 12 mois
    
    weather_df.writeTo("iceberg.bronze.weather_2024") \
              .partitionedBy("date") \
              .createOrReplace()
    
    print(f"✅ Table bronze.weather_2024 créée: {weather_df.count()} lignes")
    
    print("\n🎉 Ingestion Bronze terminée avec succès!")
    return trips_df, zones_df, weather_df

In [11]:
create_bronze_tables()

📊 Étape 1/3: Ingestion des trips taxi...
Schéma des trips:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)


📈 Distribution par année/mois:


+----+-----+-------+
|year|month|  count|
+----+-----+-------+
|2002|   12|     11|
|2008|   12|     10|
|2009|    1|     18|
|2023|   12|     10|
|2024|    1|2964617|
|2024|    2|3007533|
|2024|    3|3582611|
|2024|    4|3514295|
|2024|    5|3723843|
|2024|    6|3539170|
|2024|    7|3076876|
|2024|    8|2979192|
|2024|    9|3633025|
|2024|   10|3833780|
|2024|   11|3646372|
|2024|   12|3668350|
|2025|    1|      2|
|2025|    2|      1|
|2025|    3|      2|
|2026|    6|      2|
+----+-----+-------+



✅ Table bronze.taxi_trips_2024 créée: 41169720 lignes

🗺️  Étape 2/3: Ingestion des zones taxi...
📁 Fichier CSV non trouvé, utilisation du shapefile...
Schéma des zones:
root
 |-- OBJECTID: long (nullable = true)
 |-- Shape_Leng: double (nullable = true)
 |-- Shape_Area: double (nullable = true)
 |-- zone: string (nullable = true)
 |-- LocationID: long (nullable = true)
 |-- borough: string (nullable = true)
 |-- geometry_wkt: array (nullable = false)
 |    |-- element: string (containsNull = false)



26/01/05 12:01:34 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
                                                                                

✅ Table bronze.taxi_zones créée: 263 zones

🌤️  Étape 3/3: Ingestion des données météo...
Schéma des données météo:
root
 |-- time: timestamp (nullable = true)
 |-- temp: double (nullable = true)
 |-- rhum: integer (nullable = true)
 |-- prcp: double (nullable = true)
 |-- wspd: double (nullable = true)
 |-- pres: double (nullable = true)

Colonnes disponibles: ['time', 'temp', 'rhum', 'prcp', 'wspd', 'pres']
Aperçu des données:
+-------------------+----+----+----+----+------+
|               time|temp|rhum|prcp|wspd|  pres|
+-------------------+----+----+----+----+------+
|2024-01-01 00:00:00| 5.0|  55| 0.0| 9.0|1016.0|
|2024-01-01 01:00:00| 5.0|  55| 0.0| 9.0|1016.0|
|2024-01-01 02:00:00| 5.0|  55| 0.0|13.0|1016.0|
|2024-01-01 03:00:00| 4.0|  60| 0.0| 6.0|1016.0|
|2024-01-01 04:00:00| 4.0|  67| 0.0|10.1|1016.0|
+-------------------+----+----+----+----+------+
only showing top 5 rows

✅ Colonne 'time' convertie en 'date'


✅ Table bronze.weather_2024 créée: 8576 lignes

🎉 Ingestion Bronze terminée avec succès!


(DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double, year: int, month: int, day: int],
 DataFrame[OBJECTID: bigint, Shape_Leng: double, Shape_Area: double, zone: string, LocationID: bigint, borough: string, geometry_wkt: array<string>],
 DataFrame[time: timestamp, temp: double, rhum: int, prcp: double, wspd: double, pres: double, date: date])

In [12]:
# Vérification des tables créées
print("📋 Liste des tables dans le namespace bronze:")
spark.sql("SHOW TABLES IN iceberg.bronze").show()

# Pour une table spécifique
print("\n🔍 Description de la table taxi_trips_2024:")
spark.sql("DESCRIBE EXTENDED iceberg.bronze.taxi_trips_2024").show(truncate=False)

# Vérifier les données
print("\n📊 Aperçu des données (5 premières lignes):")
spark.sql("SELECT * FROM iceberg.bronze.taxi_trips_2024 LIMIT 5").show()

# Vérifier les partitions
print("\n🗂️ Partitions de la table taxi_trips_2024:")
spark.sql("SHOW PARTITIONS iceberg.bronze.taxi_trips_2024").show()

📋 Liste des tables dans le namespace bronze:
+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|taxi_trips_2024|      false|
|   bronze|     taxi_zones|      false|
|   bronze|   weather_2024|      false|
+---------+---------------+-----------+


🔍 Description de la table taxi_trips_2024:
+---------------------+-------------+-------+
|col_name             |data_type    |comment|
+---------------------+-------------+-------+
|VendorID             |int          |NULL   |
|tpep_pickup_datetime |timestamp_ntz|NULL   |
|tpep_dropoff_datetime|timestamp_ntz|NULL   |
|passenger_count      |bigint       |NULL   |
|trip_distance        |double       |NULL   |
|RatecodeID           |bigint       |NULL   |
|store_and_fwd_flag   |string       |NULL   |
|PULocationID         |int          |NULL   |
|DOLocationID         |int          |NULL   |
|payment_type         |bigint       |NULL   |
|fare_amount          |double   

AnalysisException: [INVALID_PARTITION_OPERATION.PARTITION_MANAGEMENT_IS_UNSUPPORTED] The partition command is invalid. Table `iceberg`.`bronze`.`taxi_trips_2024` does not support partition management.; line 1 pos 16;
ShowPartitions [partition#1229]
+- ResolvedTable org.apache.iceberg.spark.SparkCatalog@27a281de, bronze.taxi_trips_2024, iceberg.bronze.taxi_trips_2024, [VendorID#1230, tpep_pickup_datetime#1231, tpep_dropoff_datetime#1232, passenger_count#1233L, trip_distance#1234, RatecodeID#1235L, store_and_fwd_flag#1236, PULocationID#1237, DOLocationID#1238, payment_type#1239L, fare_amount#1240, extra#1241, mta_tax#1242, tip_amount#1243, tolls_amount#1244, improvement_surcharge#1245, total_amount#1246, congestion_surcharge#1247, Airport_fee#1248, year#1249, month#1250, day#1251]


In [13]:
print("\n🗂️ Partitions de la table taxi_trips_2024:")
# Pour Iceberg, on utilise des requêtes de métadonnées
spark.sql("""
    SELECT year, month, COUNT(*) as record_count 
    FROM iceberg.bronze.taxi_trips_2024 
    GROUP BY year, month 
    ORDER BY year, month
""").show()

print("\n🌡️ Partitions de la table weather_2024:")
spark.sql("""
    SELECT date, COUNT(*) as record_count 
    FROM iceberg.bronze.weather_2024 
    GROUP BY date 
    ORDER BY date
    LIMIT 10
""").show()


🗂️ Partitions de la table taxi_trips_2024:


+----+-----+------------+
|year|month|record_count|
+----+-----+------------+
|2002|   12|          11|
|2008|   12|          10|
|2009|    1|          18|
|2023|   12|          10|
|2024|    1|     2964617|
|2024|    2|     3007533|
|2024|    3|     3582611|
|2024|    4|     3514295|
|2024|    5|     3723843|
|2024|    6|     3539170|
|2024|    7|     3076876|
|2024|    8|     2979192|
|2024|    9|     3633025|
|2024|   10|     3833780|
|2024|   11|     3646372|
|2024|   12|     3668350|
|2025|    1|           2|
|2025|    2|           1|
|2025|    3|           2|
|2026|    6|           2|
+----+-----+------------+


🌡️ Partitions de la table weather_2024:


+----------+------------+
|      date|record_count|
+----------+------------+
|2024-01-01|          24|
|2024-01-02|          24|
|2024-01-03|          24|
|2024-01-04|          24|
|2024-01-05|          24|
|2024-01-06|          24|
|2024-01-07|          24|
|2024-01-08|          24|
|2024-01-09|          24|
|2024-01-10|          24|
+----------+------------+



In [14]:
print("\n🔍 Schéma complet de taxi_trips_2024:")
# Afficher toutes les colonnes sans tronquer
spark.sql("DESCRIBE iceberg.bronze.taxi_trips_2024").show(truncate=False, numRows=50)

# Vérifier les métadonnées Iceberg
print("\n📊 Métadonnées Iceberg:")
spark.sql("""
    SELECT * 
    FROM iceberg.bronze.taxi_trips_2024.snapshots 
    ORDER BY committed_at DESC 
    LIMIT 1
""").show(truncate=False)


🔍 Schéma complet de taxi_trips_2024:


TypeError: DataFrame.show() got an unexpected keyword argument 'numRows'